<a href="https://colab.research.google.com/github/Dohwon/AI-Agent-Project/blob/main/250731_%ED%95%99%EC%8A%B5%EB%8D%B0%EC%9D%B4%ED%84%B0_%ED%98%95%ED%83%9C%EC%86%8C%EB%B6%84%EC%84%9D%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 필요한 패키지 설치 (처음 한 번만 실행)

!pip install openpyxl
!pip install xlsxwriter
!apt-get install -y openjdk-11-jdk  #konlpy 위한 jdk
!pip install konlpy

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
# 실행 환경 체크
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

file_path = None
file_name = None
df = None

In [ ]:
import os
import platform

# Colab 또는 Linux에서만 JAVA_HOME 설정
def is_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if is_colab():
    # JAVA 설치 및 환경 설정 (최초 1회만)
    !apt-get install -y openjdk-11-jdk
    !pip install konlpy
    !pip install openpyxl xlsxwriter
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
else:
    print("Jupyter 환경 감지 - JAVA_HOME 설정 건너뜀")

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
openjdk-11-jdk is already the newest version (11.0.28+6-1ubuntu1~22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
from konlpy.tag import Okt
import pandas as pd
from collections import defaultdict
from IPython.display import FileLink
import ipywidgets as widgets
import openpyxl
from datetime import datetime
import sys

In [ ]:
# konlpy 설치 테스트

from konlpy.tag import Okt
okt = Okt()
print(okt.pos("형태소 분석기 테스트", stem=True))

[('형태소', 'Noun'), ('분석', 'Noun'), ('기', 'Noun'), ('테스트', 'Noun')]


In [ ]:
# 📌 학습데이터 기반 형태소 분석기
# 이 코드는 **Jupyter Notebook 및 Google Colab**에서 실행할 수 있습니다.

In [ ]:
# 파일업로드 - Colab인 경우
if is_colab():
    from google.colab import files
    uploaded = files.upload()

    if uploaded:
        file_name = list(uploaded.keys())[0]
        file_path = file_name
        df = pd.read_excel(file_name)
        print(f"✅ Colab에서 '{file_name}' 업로드 완료!")

# 파일업로드 - Jupyter인 경우
else:
    file_path = "C:/Users/dowon/Desktop/학습데이터_싱글턴.xlsx"  # ← 너의 엑셀 파일명으로 바꿔줘!
    file_name = os.path.splitext(os.path.basename(file_path))[0]
    df = pd.read_excel(file_path)

    print("Jupyter 환경: '{file_name}' 업로드 완료!")

Saving 학습데이터_싱글턴.xlsx to 학습데이터_싱글턴 (1).xlsx
✅ Colab에서 '학습데이터_싱글턴 (1).xlsx' 업로드 완료!


In [ ]:
# 1. 엑셀 파일 불러오기
file_name = os.path.splitext(os.path.basename(file_path))[0]
df = pd.read_excel(file_path)  # 첫 번째 행을 헤더로 사용하지 않음

In [ ]:
# 2. 컬럼 지정
intent_col = df.columns[0]
text_col = df.columns[1]

okt = Okt()
records = []
records_tokens = []

In [ ]:
# 3-1. 인텐트별로 그룹지어 문장 합치고 분석 - 원형 단어만 남김
intent_grouped = df.groupby(intent_col)

for intent, group in intent_grouped:
    full_text = " ".join(group[text_col].astype(str).tolist())

    tokens_stem = okt.pos(full_text, stem=True)
    tokens_raw = okt.pos(full_text, stem=False)

    i = 0
    while i < len(tokens_raw):
        word_raw, pos_raw = tokens_raw[i]
        word_stem, pos_stem = tokens_stem[i]

        if pos_raw in ["Noun", "Verb", "Adjective"]:
            combined = word_raw
            lemma = word_stem
            j = i + 1

            while j < len(tokens_raw) and tokens_raw[j][1] in ["Josa", "Eomi", "Suffix", "Verb"]:
                combined += tokens_raw[j][0]
                j += 1

            records.append({
                "인텐트": intent,
                "원형 단어": f"{lemma}({pos_raw})",
                "조합 형태": combined
            })
            i = j
        else:
            i += 1

In [ ]:
# 3-2. 인텐트별로 그룹지어 문장 합치고 분석 - 모든 품사태깅 형태를 남김
intent_grouped = df.groupby(intent_col)


for intent, group in intent_grouped:
    full_text = " ".join(group[text_col].astype(str).tolist())

    tokens = okt.pos(full_text, stem=False)

    i = 0
    for word, pos in tokens:
      records_tokens.append({
          "인텐트": intent,
          "단어(품사)": f"{word}({pos})"
      })


df_records_tokens = pd.DataFrame(records_tokens)
df_counted_tokens = df_records_tokens.groupby(["인텐트", "단어(품사)"]).size().reset_index(name="등장 수")

grouped_token_df = df_counted_tokens.sort_values(by=["인텐트", "등장 수"], ascending=[True, False])

grouped_token_df.head(10)

,인텐트,단어(품사),등장 수
211,<sk_0>()<sk_end>,청정(Noun),314
22,<sk_0>()<sk_end>,공기(Noun),211
19,<sk_0>()<sk_end>,고정(Noun),209
25,<sk_0>()<sk_end>,기능(Noun),99
212,<sk_0>()<sk_end>,청정기(Noun),84
118,<sk_0>()<sk_end>,시작(Noun),72
208,<sk_0>()<sk_end>,지금(Noun),64
134,<sk_0>()<sk_end>,실행(Noun),59
76,<sk_0>()<sk_end>,모드(Noun),58
82,<sk_0>()<sk_end>,바로(Noun),53


In [ ]:
# 4. 결과 집계 - 원형 단어 기준
result_df = pd.DataFrame(records)
result_df["등장 수"] = 1
grouped_df = result_df.groupby(["인텐트", "원형 단어", "조합 형태"], as_index=False)["등장 수"].sum()
grouped_df = grouped_df.sort_values(by=["인텐트", "등장 수"], ascending=[True, False])

grouped_df.head(10)

,인텐트,원형 단어,조합 형태,등장 수
341,<sk_0>()<sk_end>,청정(Noun),청정,267
45,<sk_0>()<sk_end>,공기(Noun),공기,211
43,<sk_0>()<sk_end>,고정(Noun),고정,209
382,<sk_0>()<sk_end>,청정기(Noun),청정기,72
48,<sk_0>()<sk_end>,기능(Noun),기능,69
327,<sk_0>()<sk_end>,지금(Noun),지금,56
114,<sk_0>()<sk_end>,바로(Noun),바로,45
337,<sk_0>()<sk_end>,청(Noun),청,39
98,<sk_0>()<sk_end>,모드(Noun),모드,34
158,<sk_0>()<sk_end>,수(Noun),수,25


In [ ]:
# 5-1. 피벗 시트 만들기 - 원문 단어
pivot_df = grouped_df.groupby(["인텐트", "원형 단어"], as_index=False)["등장 수"].sum()
pivot_table = pivot_df.pivot_table(index="원형 단어", columns="인텐트", values="등장 수", fill_value=0)
pivot_table["총합"] = pivot_table.sum(axis=1)
pivot_table = pivot_table.sort_values(by="총합", ascending=False).drop(columns="총합")
pivot_table = pivot_table.sort_index(key=lambda x: x.str.normalize("NFKD"))
pivot_table = pivot_table.astype(int) # 정수로 변환


display(pivot_table)

인텐트,<sk_0>()<sk_end>,<sk_100>(type=1)<sk_end>,<sk_100>(type=10)<sk_end>,<sk_100>(type=2)<sk_end>,<sk_100>(type=3)<sk_end>,<sk_100>(type=4)<sk_end>,<sk_100>(type=5)<sk_end>,<sk_100>(type=6)<sk_end>,<sk_100>(type=7)<sk_end>,<sk_100>(type=8)<sk_end>,...,<sk_8>(get=True)<sk_end>,<sk_8>(volume=0)<sk_end>,<sk_8>(volume=1)<sk_end>,<sk_8>(volume=2)<sk_end>,<sk_8>(volume=3)<sk_end>,<sk_8>(volume=4)<sk_end>,<sk_8>(volume=5)<sk_end>,<sk_9>(get=True)<sk_end>,<sk_9>(mute=False)<sk_end>,<sk_9>(mute=True)<sk_end>
원형 단어,,,,,,,,,,,,,,,,,,,,,
가구(Noun),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가능(Noun),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
가능하다(Adjective),2,0,0,0,0,0,3,0,0,10,...,6,0,0,0,1,0,0,5,0,2
가다(Verb),0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
가동(Noun),32,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
후(Noun),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
흔하다(Adjective),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
흡입(Noun),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# 5-2. 피벗 시트 만들기 - 원문 토큰
pivot_df = grouped_token_df.groupby(["인텐트", "단어(품사)"], as_index=False)["등장 수"].sum()
pivot_table_token = pivot_df.pivot_table(index="단어(품사)", columns="인텐트", values="등장 수", fill_value=0)
pivot_table_token["총합"] = pivot_table_token.sum(axis=1)
pivot_table_token = pivot_table_token.sort_values(by="총합", ascending=False).drop(columns="총합")
pivot_table_token = pivot_table_token.sort_index(key=lambda x: x.str.normalize("NFKD"))
pivot_table_token = pivot_table_token.astype(int) # 정수로 변환


display(pivot_table_token)

인텐트,<sk_0>()<sk_end>,<sk_100>(type=1)<sk_end>,<sk_100>(type=10)<sk_end>,<sk_100>(type=2)<sk_end>,<sk_100>(type=3)<sk_end>,<sk_100>(type=4)<sk_end>,<sk_100>(type=5)<sk_end>,<sk_100>(type=6)<sk_end>,<sk_100>(type=7)<sk_end>,<sk_100>(type=8)<sk_end>,...,<sk_8>(get=True)<sk_end>,<sk_8>(volume=0)<sk_end>,<sk_8>(volume=1)<sk_end>,<sk_8>(volume=2)<sk_end>,<sk_8>(volume=3)<sk_end>,<sk_8>(volume=4)<sk_end>,<sk_8>(volume=5)<sk_end>,<sk_9>(get=True)<sk_end>,<sk_9>(mute=False)<sk_end>,<sk_9>(mute=True)<sk_end>
단어(품사),,,,,,,,,,,,,,,,,,,,,
0(Number),0,0,0,0,0,0,0,0,0,0,...,0,22,0,0,0,0,0,0,0,0
01(Number),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
02(Number),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
03(Number),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
04(Number),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
후(Noun),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
흔히(Adjective),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
흡입(Noun),0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(file_name)

학습데이터_싱글턴 (1)


In [ ]:
# 6. 엑셀 저장 준비

# 6-1. 엑셀 저장 준비 - 오늘 날짜 가져오기 (YYYYMMDD 형식)
today_date = datetime.now().strftime("%Y%m%d")

# 6-2. 엑셀 저장 준비 - 파일 저장 함수 (덮어쓰지 않고 자동 넘버링)
def get_unique_filename(file_name):
    """
    중복되지 않는 파일명을 반환하는 함수.
    base_name: "{file_name}_인텐트명_정규화빈도_YYYYMMDD.xlsx" 형태의 기본 파일명
    """
    base_name = f"{file_name}.xlsx"  # 기본 파일명
    count = 1

    # 동일한 파일이 존재하면 (1), (2), (3) 형식으로 이름 변경
    while os.path.exists(base_name):
        base_name = f"{file_name} ({count}).xlsx"
        count += 1

    return base_name

# 6-3. 엑셀 저장 준비 - 최종 파일명 설정
base_filename = f"{file_name}_인텐트별_정규화빈도_{today_date}"  # 기본 파일명 생성
output_filename = get_unique_filename(base_filename)  # 중복되지 않는 파일명 생성

# print(output_filename)

with pd.ExcelWriter(output_filename, engine="openpyxl") as writer:
    df.to_excel(writer, sheet_name="원본 문장", index=False)
    grouped_df.to_excel(writer, sheet_name="인텐트별 정규화 단어", index=False)
    pivot_table.to_excel(writer, sheet_name="정규화 단어 피벗", index=True)
    pivot_table_token.to_excel(writer, sheet_name="전체 토큰 피벗", index=True)

print(f"✅ 엑셀 저장 완료: {output_filename}")

✅ 엑셀 저장 완료: 학습데이터_싱글턴 (1)_인텐트별_정규화빈도_20250801.xlsx


In [ ]:
# 7. 파일 다운로드

# 7-1. jupyter notebook
FileLink(output_filename)


# 7-1. Colab이면 자동 다운로드
def is_colab():
    return "google.colab" in sys.modules

# 7-1. Google Colab이면 자동 다운로드 실행
if is_colab():
    from google.colab import files
    files.download(output_filename)  # 파일 자동 다운로드
    print(f"✅ Colab에서 파일이 자동으로 다운로드되었습니다: {output_filename}")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

✅ Colab에서 파일이 자동으로 다운로드되었습니다: 학습데이터_싱글턴 (1)_인텐트별_정규화빈도_20250801.xlsx


In [ ]:
# 6-2. jupyter notebook
FileLink(output_filename)

/content/학습데이터_싱글턴 (1)_인텐트별_정규화빈도_20250801.xlsx